In [1]:
import numpy as np
import pandas as pd
import mlflow

In [2]:
mlflow.set_tracking_uri('http://ec2-51-20-64-139.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('exp 5 - ML Algos with HP Tuning')

<Experiment: artifact_location='s3://neeraj-first-bucket/615931187569087796', creation_time=1737545423333, experiment_id='615931187569087796', last_update_time=1737545423333, lifecycle_stage='active', name='exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
import optuna

c:\data\projects\comment_analysis\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

# vectorizer
ngram_range = (1,2)
max_features = 2000
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
X_train_trf = vectorizer.fit_transform(X_train)
X_test_trf = vectorizer.fit_transform(X_test)

# oversampling
adasyn = ADASYN(random_state=42)
X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)


In [6]:
def log_mlflow(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run(nested=True):
        mlflow.set_tag("mlflow.runName", f"{model_name}_Adasyn_Tfidf_Bigram")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algorithm", model_name)

        # model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        mlflow.log_metric("accoracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))

In [8]:
def optuna_run_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_svm, n_trials=30)

    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)
    
    log_mlflow("SVM", best_model, X_train_trf, y_train, X_test_trf, y_test)

In [9]:
optuna_run_experiment()

[I 2025-01-23 16:14:44,947] A new study created in memory with name: no-name-2082bdee-6f76-4348-ad07-16a35cdbebb3
[W 2025-01-23 16:33:49,682] Trial 0 failed with parameters: {'C': 0.6090647226738057, 'kernel': 'poly'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\data\projects\comment_analysis\myenv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Appsquadz\AppData\Local\Temp\ipykernel_31924\3391492586.py", line 6, in objective_svm
    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))
                                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "c:\data\projects\comment_analysis\myenv\Lib\site-packages\sklearn\svm\_base.py", line 821, in predict
    y = super().predict(X)
  File "c:\data\projects\comment_analysis\myenv\Lib\site-packages\sklearn\svm\_base.py", line 437, in predict
    return predict(

KeyboardInterrupt: 